In [1]:
import numpy as np
import pandas as pd
import bs4
import requests
import nltk
from nltk.corpus import stopwords
from collections import Counter 

In [2]:
# Scraping List of 300 Jobs
s1 = '&start='
url_loop_str = ['']
for i in np.arange(25,300,25):
    url_loop_str.append(s1+str(i))

soup = bs4.BeautifulSoup()

for i in url_loop_str:

    base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
    keyword = "Data%20Science"
    location = "United%20States"
    refresh = "true"
    
    soup_list = []

    url = f"{base_url}?keywords={keyword}&location={location}&refresh={refresh}{i}"
    page = requests.get(url)

    recepie = bs4.BeautifulSoup(page.text,"html")

    soup.append(recepie) 

In [3]:
def get(soup):

    role = []
    c_name = []
    dt = []
    loc = []
    j_url = []
    c_url = []
    ind = ''
    
    n_applicants = []
    salary_range = []

    seniority_level = []
    employement_type = []
    job_function = []
    industry = []

    about_info = []

    raw_list = soup.find_all('li')

    for i in range(len(raw_list)):
        
        # Position --------------------------------------------
        role.append(raw_list[i].h3.contents[0].strip())
        
        # Company Name
        if raw_list[i].h4.a:
            c_name.append(raw_list[i].h4.a.contents[0].strip())
        else: 
            c_name.append(raw_list[i].h4.contents[0].strip())
        
        # Job Location ----------------------------------------
        if raw_list[i].find(class_ = 'job-search-card__location').contents[0].strip():
            loc.append(raw_list[i].find(class_ = 'job-search-card__location').contents[0].strip())
        else: 
            loc.append('')
        
        # Job Posting Date ------------------------------------
        if raw_list[i].find(class_ = 'job-search-card__listdate'):
            dt.append(raw_list[i].find(class_ = 'job-search-card__listdate').contents[0].strip())
        else: 
            dt.append(raw_list[i].find(class_ = 'job-search-card__listdate--new').contents[0].strip())

        # Job Post URL ----------------------------------------
        if raw_list[i].find_all('a', href=True)[0]:
            j_url.append(raw_list[i].find_all('a', href=True)[0]['href'])
        else: 
            j_url.append('') 
        
        if raw_list[i].find_all('a', href=True)[0]:
            n_applicants_var, salary_range_var, seniority_level_var, employement_type_var, job_function_var, industry_var, about_info_var = extract_job_attr(raw_list[i].find_all('a', href=True)[0]['href'])
        else:
            n_applicants_var, salary_range_var, seniority_level_var, employement_type_var, job_function_var, industry_var, about_info_var = '', '', '', '', '', '', ''
        
        n_applicants.append(n_applicants_var)
        salary_range.append(salary_range_var)

        seniority_level.append(seniority_level_var)
        employement_type.append(employement_type_var)
        job_function.append(job_function_var)
        industry.append(industry_var)

        about_info.append(about_info_var)

        # Company URL ----------------------------------------
        if len(raw_list[i].find_all('a', href=True)) == 2:
            c_url.append(raw_list[i].find_all('a', href=True)[1]['href'])
        else: 
            c_url.append('')

    df = pd.DataFrame(list(zip(role, c_name, loc, dt, j_url, c_url, 
                               n_applicants, salary_range, 
                               seniority_level, employement_type, job_function, industry, 
                               about_info)),
                    columns =['JOB_ROLE', 'COMPANY', 'LOCATION', 'POST_DATE', 'JOB_URL', 'COMPANY_URL',
                              'N_APPLICANTS', 'SALARY_RANGE', 
                              'SENIORITY_LEVEL', 'EMPLOYEMENT_TYPE', 'JOB_FUNCTION', 'INDUSTRY',
                              'ABOUT'])

    df = df[~df['JOB_ROLE'].str.upper().str.contains('INTERN')]

    return df  

In [4]:
# Getting Job Attributes
def extract_job_attr(url):
    if url == '':
        return '', '', '', '', '', '', ''
    
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.text, "html.parser")

    n_applicants = ''
    salary_range = ''

    seniority_level = ''
    employement_type = ''
    job_function = ''
    industry = ''
    about_info = ''

    # No. Applicants ----------------------------------------
    if soup.find(class_ = 'num-applicants__caption'):
        n_applicants = soup.find(class_ = 'num-applicants__caption').contents[0].strip()
    else: 
        n_applicants = ''
        
    # Compensation ----------------------------------------
    if soup.find(class_ = 'salary compensation__salary'):
        salary_range = soup.find(class_ = 'salary compensation__salary').contents[0].strip()
    else: 
        salary_range = ''

    # Job Attributes --------------------------------------
    job_attr = soup.find_all(class_ = 'description__job-criteria-text description__job-criteria-text--criteria')
    if job_attr:
        seniority_level = job_attr[0].contents[0].strip() if job_attr[0].contents else ''
        employement_type = job_attr[1].contents[0].strip() if job_attr[1].contents else ''
        job_function = job_attr[2].contents[0].strip() if job_attr[2].contents else ''
        industry = job_attr[3].contents[0].strip() if job_attr[3].contents else ''
    else:
        seniority_level = ''
        employement_type = ''
        job_function = ''
        industry = ''
    
    # About ----------------------------------------------
    if soup.find(class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden'):
        for i in soup.find(class_ = 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden').contents:
            about_info += str(i)
    
    return n_applicants, salary_range, seniority_level, employement_type, job_function, industry, about_info

In [5]:
data = get(soup)
data.head() 

,JOB_ROLE,COMPANY,LOCATION,POST_DATE,JOB_URL,COMPANY_URL,N_APPLICANTS,SALARY_RANGE,SENIORITY_LEVEL,EMPLOYEMENT_TYPE,JOB_FUNCTION,INDUSTRY,ABOUT
11,"Data Scientist, GBG Data Science",Meta,"New York, NY",1 week ago,https://www.linkedin.com/jobs/view/data-scient...,https://www.linkedin.com/company/meta?trk=publ...,98 applicants,"$134,000.00/yr - $194,000.00/yr",Not Applicable,Full-time,Engineering and Information Technology,"Technology, Information and Internet",\n The massive scale and heavy engageme...
12,"Data Scientist, GBG Data Science",Meta,"New York, NY",1 week ago,https://www.linkedin.com/jobs/view/data-scient...,https://www.linkedin.com/company/meta?trk=publ...,94 applicants,"$134,000.00/yr - $194,000.00/yr",Not Applicable,Full-time,Engineering and Information Technology,"Technology, Information and Internet",\n The massive scale and heavy engageme...
14,"Data Scientist, Marketing",Asana,"San Francisco, CA",2 weeks ago,https://www.linkedin.com/jobs/view/data-scient...,https://www.linkedin.com/company/asana?trk=pub...,Over 200 applicants,"$171,000.00/yr - $258,000.00/yr",Mid-Senior level,Full-time,Engineering,Software Development,\n<p>Asana's Data Science team helps us fulfil...
15,ML Engineer (Entry Level),Grindr,"San Francisco, CA",1 week ago,https://www.linkedin.com/jobs/view/ml-engineer...,https://www.linkedin.com/company/grindr?trk=pu...,,,,,,,
16,Data Analyst,Intel Corporation,"Phoenix, AZ",1 week ago,https://www.linkedin.com/jobs/view/data-analys...,https://www.linkedin.com/company/intel-corpora...,,,,,,,


In [13]:
def get_freq(data):
    
    nltk.download('stopwords')
    
    s1 = ''
    rm_list = ['*','%','[',']','#','%','•','©','®','+','</ol>','<ol>','<span>','</span>','<u>','</u>']
    for i,j in data.ABOUT.iteritems():
        s1 += j

    s1 = s1.replace('<br/>',' ').replace('</strong>',' ').replace('<strong>', ' ').replace('</li>', ' ').replace('<li>', ' ').replace('<ul>', ' ').replace('</ul>', ' ').replace('<em>', ' ').replace('</em>', ' ').replace('</p>', ' ').replace('<p>', ' ').replace(',', ' ').replace('.',' ').replace('(',' ').replace(')',' ').replace("’",' ').lower()

    for i in rm_list:
        s1 = s1.replace(i, ' ')

    s2 = s1.split(' ') 

    s3 = [i for i in s2 if i not in stopwords.words('english')]

    res = dict(Counter(s3))

    res2 = {}
    for w in sorted(res, key=res.get, reverse=True):
        res2[w] = res[w]
    return pd.DataFrame(res2.items())

frequency_analysis = get_freq(data)
frequency_analysis 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aakashrathod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/mn/89jgthfx1f7b2mlbmf36w11c0000gn/T/ipykernel_61315/466306341.py:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i,j in data.ABOUT.iteritems():


,0,1
0,,20699
1,data,1640
2,experience,645
3,work,419
4,learning,356
...,...,...
6411,existed,1
6412,crave,1
6413,post-event,1
6414,segmented,1


In [14]:
writer = pd.ExcelWriter('LinkedIn Job DS.xlsx', engine = 'xlsxwriter')

data.to_excel(writer, sheet_name= 'Jobs', index = False)
frequency_analysis.to_excel(writer, sheet_name= 'Frequency', index = False)

writer.close() 

# The END